<a href="https://colab.research.google.com/github/teohangxanh/5290/blob/master/Solution_Text_Classification_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%matplotlib inline
!pip install torch>=1.3.1
!pip install torchtext==0.4

In [3]:
import torch
import torchtext
#from torchtext.legacy.datasets import text_classification
from torchtext.datasets import text_classification

# import os
# if not os.path.isdir('./.data'):
#     os.mkdir('./.data')
# train_dataset, test_dataset = text_classification.DATASETS['AG_NEWS'](
#     root='./.data', vocab=None)
# BATCH_SIZE = 16

120000lines [00:10, 11594.28lines/s]
120000lines [00:20, 5960.42lines/s]
7600lines [00:01, 6138.00lines/s]


In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
import torch
import torchtext
from torchtext import data

In [6]:
max_seq_len = 50
TEXT = data.Field(tokenize="spacy", batch_first=True, include_lengths=True, fix_length=max_seq_len)
LABEL = data.LabelField(dtype=torch.float, batch_first=True)

In [7]:
fields = [('label', LABEL), (None, None), ('text',TEXT)]

In [8]:
%ls drive/MyDrive/UNT/AG_news

ls: cannot access 'drive/MyDrive/UNT/AG_news': No such file or directory


In [14]:
training_data=data.TabularDataset(path = 'drive/MyDrive/train.csv',format = 'csv',fields = fields,skip_header = True)

In [15]:
print(vars(training_data.examples[0]))

{'label': '3', 'text': ['Reuters', '-', 'Private', 'investment', 'firm', 'Carlyle', 'Group,\\which', 'has', 'a', 'reputation', 'for', 'making', 'well', '-', 'timed', 'and', 'occasionally\\controversial', 'plays', 'in', 'the', 'defense', 'industry', ',', 'has', 'quietly', 'placed\\its', 'bets', 'on', 'another', 'part', 'of', 'the', 'market', '.']}


In [16]:
train_data, valid_data = training_data.split(split_ratio=0.1)

In [17]:
#initialize glove embeddings
TEXT.build_vocab(train_data,min_freq=3,vectors = "glove.6B.300d")  
LABEL.build_vocab(train_data)

#No. of unique tokens in text
print("Size of TEXT vocabulary:",len(TEXT.vocab))

#No. of unique tokens in label
print("Size of LABEL vocabulary:",len(LABEL.vocab))

#Commonly used words
print(TEXT.vocab.freqs.most_common(10))  

#Word dictionary
# print(TEXT.vocab.stoi)

.vector_cache/glove.6B.zip: 862MB [02:39, 5.40MB/s]                           
100%|█████████▉| 399999/400000 [00:47<00:00, 8491.95it/s]


Size of TEXT vocabulary: 2261
Size of LABEL vocabulary: 4
[('the', 1699), (',', 1448), ('.', 1230), ('-', 1002), ('a', 900), ('to', 900), ('of', 871), ('in', 742), ('and', 589), (' ', 460)]


In [18]:
batch_size = 64

train_iterator, valid_iterator = data.BucketIterator.splits((train_data, valid_data), batch_size=batch_size,
                                                           sort_key=lambda x: len(x.text),
                                                           sort_within_batch=True)

In [19]:
# Create neural network representation
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

class CNNTextClassification(nn.Module):
    def __init__(self, vocabulary_size, embedding_size, max_seq_len, out_channels,
                 kernel_heights, dropout, num_class):
        super().__init__()
        self.out_channels = out_channels
        self.kernel_heights = kernel_heights
        self.embedding_size = embedding_size
        self.max_seq_len = max_seq_len
        
        self.embedding = nn.Embedding(vocabulary_size, embedding_size)
        
        self.conv1 = nn.Sequential(nn.Conv1d(in_channels=self.embedding_size, out_channels=self.out_channels,
                               kernel_size=self.kernel_heights[0]),
                                   nn.ReLU(),
                                  nn.MaxPool1d(self.max_seq_len - self.kernel_heights[0]+1))
        
        self.conv2 = nn.Sequential(nn.Conv1d(in_channels=self.embedding_size, out_channels=self.out_channels,
                               kernel_size=self.kernel_heights[1]),
                                   nn.ReLU(),
                                  nn.MaxPool1d(self.max_seq_len - self.kernel_heights[1]+1))
        
        self.conv3 = nn.Sequential(nn.Conv1d(in_channels=self.embedding_size, out_channels=self.out_channels,
                               kernel_size=self.kernel_heights[2]),
                                   nn.ReLU(),
                                  nn.MaxPool1d(self.max_seq_len - self.kernel_heights[2]+1))
        
        self.dropout = nn.Dropout(dropout)
        
        self.fc = nn.Linear(len(self.kernel_heights) * out_channels, num_class)
        
        self.softmax = nn.LogSoftmax(dim=-1)
        
    def forward(self, text, text_lengths):
        emb = self.embedding(text).permute(0, 2, 1)
        
        conv_out1 = self.conv1(emb).squeeze(2)
        conv_out2 = self.conv2(emb).squeeze(2)
        conv_out3 = self.conv3(emb).squeeze(2)
        
        all_out = torch.cat((conv_out1, conv_out2, conv_out3), 1)
        final_feature_map = self.dropout(all_out)
        
        final_out = self.fc(final_feature_map)
        
        return self.softmax(final_out)

In [20]:
vocabulary_size = len(TEXT.vocab)
n_class = len(LABEL.vocab)
embedding_size = 300
out_channels = 100
kernel_heights = [3, 4, 5]
dropout = 0.4

model = CNNTextClassification(vocabulary_size, embedding_size, max_seq_len,
                              out_channels, kernel_heights, dropout, n_class)

In [21]:
model

CNNTextClassification(
  (embedding): Embedding(2261, 300)
  (conv1): Sequential(
    (0): Conv1d(300, 100, kernel_size=(3,), stride=(1,))
    (1): ReLU()
    (2): MaxPool1d(kernel_size=48, stride=48, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv1d(300, 100, kernel_size=(4,), stride=(1,))
    (1): ReLU()
    (2): MaxPool1d(kernel_size=47, stride=47, padding=0, dilation=1, ceil_mode=False)
  )
  (conv3): Sequential(
    (0): Conv1d(300, 100, kernel_size=(5,), stride=(1,))
    (1): ReLU()
    (2): MaxPool1d(kernel_size=46, stride=46, padding=0, dilation=1, ceil_mode=False)
  )
  (dropout): Dropout(p=0.4, inplace=False)
  (fc): Linear(in_features=300, out_features=4, bias=True)
  (softmax): LogSoftmax(dim=-1)
)

In [22]:
pretrained_embeddings = TEXT.vocab.vectors
model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0466,  0.2132, -0.0074,  ...,  0.0091, -0.2099,  0.0539],
        ...,
        [ 0.3316,  0.2792, -0.1181,  ..., -0.4395,  0.2438, -0.0445],
        [ 0.3398,  0.0483,  0.4093,  ...,  0.2935, -0.0009,  0.0287],
        [ 0.2249, -0.1571,  0.0025,  ..., -0.2524,  0.0694,  0.3659]])

In [23]:
def training(model, iterator, optimizer, criterion):
    training_loss = 0
    training_accuracy = 0
    
    model.train()
    
    for batch in iterator:
        optimizer.zero_grad()
        
        text, text_lengths = batch.text
        target = batch.label
        target = torch.autograd.Variable(target).long()
        
        output = model(text, text_lengths).squeeze()
        
        loss = criterion(output, target)
        
        training_loss += loss.item()
        
        loss.backward()
        optimizer.step()
        
        num_corrects = (torch.max(output, 1)[1].view(target.size()).data == target.data).float().sum()

        acc = num_corrects/len(batch)
        training_accuracy += acc.item()

    scheduler.step()
    
    return training_loss / len(iterator), training_accuracy / len(iterator)

def testing(model, iterator, optimizer, criterion):
    testing_loss = 0
    testing_accuracy = 0
    model.eval()
    
    for batch in iterator:
        text, text_lengths = batch.text
        target = batch.label
        target = torch.autograd.Variable(target).long()
        
        with torch.no_grad():
            output = model(text, text_lengths).squeeze()
            loss = criterion(output, target)
            
            testing_loss += loss.item()
            num_corrects = (torch.max(output, 1)[1].view(target.size()).data == target.data).float().sum()
            acc = num_corrects/len(batch)
        
            testing_accuracy += acc.item()
            
    return testing_loss / len(iterator), testing_accuracy / len(iterator)

In [26]:
import time

n_epochs = 15
min_val_loss = float("inf")
path='drive/MyDrive/saved_weights_cnn.pt'

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.3)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1, gamma=0.9)

for epoch in range(n_epochs):
    start_time = time.time()
    
    train_loss, train_acc = training(model, train_iterator, optimizer, criterion)
    val_loss, val_acc = testing(model, valid_iterator, optimizer, criterion)
    
    secs = int(time.time() - start_time)
    mins = secs / 60
    secs = secs % 60

    print('Epoch: %d' %(epoch + 1), " | time in %d minutes, %d seconds" %(mins, secs))
    print(f'\tLoss: {train_loss:.4f}(train)\t|\tAcc: {train_acc * 100:.2f}%(train)')
    print(f'\tLoss: {val_loss:.4f}(valid)\t|\tAcc: {val_acc * 100:.2f}%(valid)')
    
    if val_loss < min_val_loss:
        min_val_loss = val_loss
        torch.save(model.state_dict(), path)

IndexError: ignored

In [27]:
testing_data=data.TabularDataset(path = 'drive/MyDrive/test.csv',format = 'csv',fields = fields,skip_header = True)

In [28]:
testing_iterator = data.BucketIterator(testing_data, batch_size=batch_size,
                                                           sort_key=lambda x: len(x.text),
                                                           sort_within_batch=True)

In [29]:
model.load_state_dict(torch.load(path))

def predict(model, iterator):
    testing_accuracy = 0
    model.eval()
    
    for batch in iterator:
        text, text_lengths = batch.text
        # text = TEXT.preprocess(text)
        label = batch.label
        target = torch.autograd.Variable(label).long()
        with torch.no_grad():
            output = model(text, text_lengths).squeeze()
            num_corrects = (torch.max(output, 1)[1].view(target.size()).data == target.data).float().sum()
            acc = num_corrects / len(batch)
            testing_accuracy += acc.item()
    
    return testing_accuracy / len(iterator)

FileNotFoundError: ignored

In [ ]:
test_acc = predict(model, testing_iterator)
print(f"Accuracy {test_acc * 100:.2f}")